In [1]:
%run data.ipynb    # load data

In [2]:
import math
import sympy as sy

In [3]:
def gibbs_energy_i(i_name, T):
    x_t = T / 10000
    data = bank_td[i_name]
    phi = data['f1'] + data['f2'] * math.log(x_t) + data['f3'] / (x_t ** 2) + data['f4'] / x_t \
        + data['f5'] * x_t + data['f6'] * x_t ** 2 + data['f7'] * x_t ** 3
    gibbs = data['H'] - phi * T
    return gibbs

def G0(name, T):
    return gibbs_energy_i(name, T)

In [4]:
def calculate_delta_r1(T):
    return 2 * G0('Al', T) + 2 * G0('HCl', T) - 2 * G0('AlCl', T) - G0('H2', T)

def calculate_delta_r2(T):
    return G0('Al', T) + 2 * G0('HCl', T) - G0('AlCl2', T) - G0('H2', T)

def calculate_delta_r3(T):
    return 2 * G0('Al', T) + 6 * G0('HCl', T) - 2 * G0('AlCl3', T) - 3 * G0('H2', T)

In [5]:
Pa = 100000

def K1(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r1(T) / (R * T)) / Pa

def K2(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r2(T) / (R * T))

def K3(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r3(T) / (R * T)) / Pa



In [6]:
def eps(name):
    return math.sqrt(bank_td[name]['epsil'] * bank_td['N2']['epsil'])

def mu(name):
    return 2 * bank_td[name]['mu'] * bank_td['N2']['mu'] / (bank_td[name]['mu'] + bank_td['N2']['mu'])

def omega(name, T):
    return 1.074 * ((T / eps(name)) ** (-0.1604))

def sigma(name):
    return (bank_td[name]['sigma'] + bank_td['N2']['sigma']) / 2

def D(name, T):
    bott = Pa * sigma(name) * omega(name, T) * math.sqrt(mu(name))
    return 0.02628 * (T ** 1.5) / bott

In [7]:
T = 623.15
print(K1(T))
print(K2(T))
print(K3(T))

7.497131271125371e-07
7.221267004644182e-08
4.062199476332757e-49


In [8]:
print(D('AlCl', T))
print(D('AlCl2', T))
print(D('AlCl3', T))
print(D('HCl', T))
print(D('H2', T))

0.000191129931554728
0.00014747988098603198
0.00015245155020045148
0.0002736394738546893
0.0008762807050502889


In [9]:
PG = {
   'AlCl':0,
   'AlCl2':0,
   'AlCl3':0,
   'H2':0,
   'HCl':10000,
   'N2':90000
}

T = 623.15
HCl, AlCl, H2, AlCl2, AlCl3 = sy.symbols('HCl AlCl H2 AlCl2 AlCl3')
X = sy.Matrix([
    HCl**2 - K1(T) * AlCl**2 * H2,
    HCl**2 - K2(T) * AlCl2 * H2,
    HCl**6 - K3(T) * AlCl3**2 * H2**3,
    D('HCl', T) * (PG['HCl'] - HCl) + 2 * D('H2', T) * (PG['H2'] - H2),
    D('AlCl', T) * (PG['AlCl'] - AlCl) + 2 * D('AlCl2', T) * (PG['AlCl2'] - AlCl2) + \
        3 * D('AlCl3', T) * (PG['AlCl3'] - AlCl3) + D('HCl', T) * (PG['HCl'] - HCl)
])
Y = [HCl, AlCl, H2, AlCl2, AlCl3]
X.jacobian(Y)
# def newton_method(funcs):

Matrix([
[                2*HCl, -1.49942625422507e-6*AlCl*H2,         -7.49713127112537e-7*AlCl**2,                       0,                                 0],
[                2*HCl,                            0,           -7.22126700464418e-8*AlCl2, -7.22126700464418e-8*H2,                                 0],
[             6*HCl**5,                            0, -1.21865984289983e-48*AlCl3**2*H2**2,                       0, -8.12439895266551e-49*AlCl3*H2**3],
[-0.000273639473854689,                            0,                 -0.00175256141010058,                       0,                                 0],
[-0.000273639473854689,        -0.000191129931554728,                                    0,   -0.000294959761972064,             -0.000457354650601354]])

In [10]:
def new_zip(x, y):
    ans = []
    for i in range(len(x)):
        ans.append((x[i], y[i]))
    return ans

def equalVector(a, b, eps):
    return math.sqrt(sum([(a[i] - b[i])**2 for i in range(len(a))])) < eps

def newton_method(Funcs, Vars, x0, eps):
    J = Funcs.jacobian(Vars)
    xk1 = sy.zeros(len(x0), 1)
    xk = x0
    while not equalVector(xk, xk1, eps):
        xx = new_zip(Vars, xk)
        Jk = J.subs(xx)
        var = (Jk.T).inv().dot(Funcs.subs(xx))
        for i in range(len(xk)):
            xk1[i] = xk[i] - var[i]
        xk, xk1 = xk1, xk
    return xk
newton_method(X, Y, [1, 1, 1, 1, 1] , 0)

/home/kranya/.local/lib/python3.6/site-packages/sympy/matrices/matrices.py:1400: SymPyDeprecationWarning: 

Dot product of non row/column vectors has been deprecated since SymPy
1.2. Use * to take matrix products instead. See
https://github.com/sympy/sympy/issues/13815 for more info.

  useinstead="* to take matrix products").warn()


[1, 1, 1, 1, 1]
Matrix([[-95395.0385063783], [13437839.6868738], [-4447479.77407682], [58.7083636141654], [5981.43039713831]])
[-1118603.57944088, -2.33294369555692e+19, -4449962.13735161, 4.30012897017141e+32, -5552694.06351861]
Matrix([[888037956312.586], [-2.54556511650939e+29], [-1.17449811324533e+23], [4.51030437170653e+57], [-2.77326362774091e+32]])
[8.53316084828157e+26, 1.55410445905394e+35, -1.17449811324533e+23, 2.08514910187437e+79, 4.46872747890754e+54]
Matrix([[1.99432530400356e+48], [9.93169538304465e+84], [2.20873754855762e+25], [2.20277188430347e+164], [2.85579049181219e+104]])
[-1.06815536515300e+78, 3.95542423254637e+132, -2.59664818898937e+28, -1.79777307987078e+274, -2.13888686382291e+154]
Matrix([[2.18892103851285e+140], [-7.18523413129496e+203], [1.74292722897235e+79], [-5.31402862505418e+473], [-4.56777460676350e+228]])
[7.34139078151828e+408, -2.86644280054141e+375, 1.74292722897235e+79, -1.34889558675997e+829, 1.22313153606541e+451]
Matrix([[-5.60741530248925e+

NonInvertibleMatrixError: Matrix det == 0; not invertible.